In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import re
pd.options.display.max_columns = 100

* 待做欄位：
    * 目標變數(7,14,30,60,180天銷量,比例)
    * 該品類(7,14,30,60,180天銷量,比例) 最大值 最小值 中位數 平均數
    * 該父品類(7,14,30,60,180天銷量,比例) 最大值 最小值 中位數 平均數
* 已完成欄位：
    * [ 'business_hour_guid',
       'installment','item_name', 'item_orig_price', 'item_price',
       'item_default_daily_amount', 
       'quantity_multiplier',
       'installment_3months', 'installment_6months', 'installment_12months',
       'deny_installment', 'shippingdate','freight_amount', 'enable_isp',
       'code', 'parent_code', 'product_launch_month', 'product_launch_hour',
       'product_launch_weekday', 'campagin_period','presales', 'product_launch_year', 
       'list', 'unit_price', 'longTern',
       'category_unit_price', 'compare_cate_unit_price', 'guid_subtotal',
       'quantity_sold_guid', 'subtotal_allguid', 'quantity_sold_allguid',
       'avg_unit_price', 'len_list']

In [32]:
orderdf=pd.read_csv('0614order_df.csv')
orderdf.head(3)

,business_hour_guid,user_id,subtotal,create_time,business_hour_order_time_s,business_hour_order_time_e,business_hour_deliver_time_s,business_hour_deliver_time_e,item_orig_price,item_price,quantity_multiplier,code,parent_code
0,732103DD-F831-4210-8449-6B1917AA37F5,1111761039,699.0,2015-01-01 12:12:44.947,2015-01-01 12:00:00.000,2015-01-22 12:00:00.000,2015-01-01 00:00:00.000,2015-01-27 00:00:00.000,1350.0,699.0,90.0,413.0,412.0
1,E4849B75-FC79-4B2E-AF00-DF82E59B2872,1112303774,594.0,2015-01-01 12:36:05.753,2015-01-01 12:00:00.000,2015-01-14 12:00:00.000,2015-01-01 00:00:00.000,2015-01-23 00:00:00.000,1794.0,594.0,6.0,166.0,166.0
2,61B117B3-9E4C-4809-9992-0F89E740E5E9,1111750722,469.0,2015-01-01 12:42:11.777,2015-01-01 12:00:00.000,2015-01-02 12:00:00.000,2015-01-01 00:00:00.000,2015-01-13 00:00:00.000,1800.0,469.0,NaN,1604.0,166.0


In [33]:
orderdf=orderdf.drop(columns=['business_hour_order_time_e','business_hour_deliver_time_s','business_hour_deliver_time_e'])

In [34]:
orderdf['quantity_multiplier']=orderdf['quantity_multiplier'].fillna(1)
orderdf['unit_price']=orderdf['item_price']/orderdf['quantity_multiplier']
orderdf['order_quantity']=orderdf['subtotal']/orderdf['unit_price']
orderdf.head()

,business_hour_guid,user_id,subtotal,create_time,business_hour_order_time_s,item_orig_price,item_price,quantity_multiplier,code,parent_code,unit_price,order_quantity
0,732103DD-F831-4210-8449-6B1917AA37F5,1111761039,699.0,2015-01-01 12:12:44.947,2015-01-01 12:00:00.000,1350.0,699.0,90.0,413.0,412.0,7.766667,90.0
1,E4849B75-FC79-4B2E-AF00-DF82E59B2872,1112303774,594.0,2015-01-01 12:36:05.753,2015-01-01 12:00:00.000,1794.0,594.0,6.0,166.0,166.0,99.000000,6.0
2,61B117B3-9E4C-4809-9992-0F89E740E5E9,1111750722,469.0,2015-01-01 12:42:11.777,2015-01-01 12:00:00.000,1800.0,469.0,1.0,1604.0,166.0,469.000000,1.0
3,40E89933-05B0-4A05-8085-3E05F1BCA7DB,1111140083,599.0,2015-01-01 12:51:42.933,2015-01-01 12:00:00.000,990.0,599.0,1.0,2.0,1.0,599.000000,1.0
4,4429EC48-7928-4DFA-8909-F238D63B4062,1112012070,449.0,2015-01-01 13:00:12.467,2015-01-01 12:00:00.000,1198.0,449.0,2.0,166.0,166.0,224.500000,2.0


In [35]:
orderdf['business_hour_order_time_s']=pd.to_datetime(orderdf['business_hour_order_time_s'])
orderdf['create_time']=pd.to_datetime(orderdf['create_time'])
orderdf['day_diff']=(orderdf['create_time']-orderdf['business_hour_order_time_s']).apply(lambda x:x.days)

In [67]:
orderdf=orderdf.fillna(0)

In [68]:
#備份一下
orderDataframe=orderdf

In [70]:
orderdf.head()

,business_hour_guid,user_id,subtotal,order_quantity,code,parent_code,day_diff
0,732103DD-F831-4210-8449-6B1917AA37F5,1111761039,699.0,90.0,413.0,412.0,0
1,E4849B75-FC79-4B2E-AF00-DF82E59B2872,1112303774,594.0,6.0,166.0,166.0,0
2,61B117B3-9E4C-4809-9992-0F89E740E5E9,1111750722,469.0,1.0,1604.0,166.0,0
3,40E89933-05B0-4A05-8085-3E05F1BCA7DB,1111140083,599.0,1.0,2.0,1.0,0
4,4429EC48-7928-4DFA-8909-F238D63B4062,1112012070,449.0,2.0,166.0,166.0,0


In [71]:
orderdf=orderdf[['business_hour_guid','user_id','subtotal','order_quantity','code','parent_code','day_diff']]
orderdf[orderdf.business_hour_guid=='732103DD-F831-4210-8449-6B1917AA37F5'].head(3)

,business_hour_guid,user_id,subtotal,order_quantity,code,parent_code,day_diff
0,732103DD-F831-4210-8449-6B1917AA37F5,1111761039,699.0,90.0,413.0,412.0,0
61,732103DD-F831-4210-8449-6B1917AA37F5,1111471636,699.0,90.0,413.0,412.0,0
219,732103DD-F831-4210-8449-6B1917AA37F5,1111129898,699.0,90.0,413.0,412.0,0


In [72]:
buiness_hour_guidlist=orderdf.business_hour_guid.unique()

In [73]:
temptable=orderdf.drop_duplicates('business_hour_guid')[['business_hour_guid']]
temptable.head(3)

,business_hour_guid
0,732103DD-F831-4210-8449-6B1917AA37F5
1,E4849B75-FC79-4B2E-AF00-DF82E59B2872
2,61B117B3-9E4C-4809-9992-0F89E740E5E9


待做欄位：
目標變數(7,14,30,60,180天銷量,比例)V
該品類(7,14,30,60,180天銷量,比例) 最大值 最小值 中位數 平均數
該父品類(7,14,30,60,180天銷量,比例) 最大值 最小值 中位數 平均數

## 目標變數

In [59]:
total_subtotal= orderdf.groupby('business_hour_guid')['subtotal'].sum().to_frame() #總銷售金額依檔次分組
total_subtotal= total_subtotal.rename(columns={'subtotal':'total_sales_amount'}).reset_index(drop=False)
total_subtotal.head(3)

,business_hour_guid,total_sales_amount
0,00000793-A001-4C8C-9196-3E6971181246,399.0
1,00001911-C3DF-4143-BC25-8AA5FA1A9AF4,1150.0
2,00001928-AE75-4AA9-8A47-4BDE96D08C2D,23040.0


In [60]:
output=orderdf
list = [7,14,30,60,180]
sales_amount_ratio=pd.DataFrame()
for i in list:
    if(i==7):
        output_days=output[output.day_diff<i]  
        text=("sales_"+str(i)+"days")
        text2=("ratio_"+str(i)+"days")
        final_output = output_days.groupby('business_hour_guid')['subtotal'].sum().to_frame()  #i天數內之銷售金額
        final_output = final_output.rename(columns={'subtotal':text})  #更改欄位名稱
        total = pd.merge(total_subtotal,final_output ,how='left',on=['business_hour_guid'])
        total = total.fillna(0)   #補空值
        total[text2] = (total[text]/total['total_sales_amount']).round(3) #銷量占比
        sales_amount_ratio = total.reset_index(drop=True) 
    else:
        output_days=output[output.day_diff<i]  
        text=("sales_"+str(i)+"days")
        text2=("ratio_"+str(i)+"days")
        final_output = output_days.groupby('business_hour_guid')['subtotal'].sum().to_frame()  
        final_output = final_output.rename(columns={'subtotal':text})  
        final_output=final_output.reset_index() 
        total = pd.merge(total_subtotal,final_output ,how='left',on=['business_hour_guid'])
        total = total.fillna(0)   
        total[text2] = (total[text]/total['total_sales_amount']).round(3) 
        sales_amount_ratio=sales_amount_ratio.merge(total,how='left')

In [61]:
sales_amount_ratio.head(3)

,business_hour_guid,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days
0,00000793-A001-4C8C-9196-3E6971181246,399.0,399.0,1.0,399.0,1.0,399.0,1.0,399.0,1.0,399.0,1.0
1,00001911-C3DF-4143-BC25-8AA5FA1A9AF4,1150.0,1150.0,1.0,1150.0,1.0,1150.0,1.0,1150.0,1.0,1150.0,1.0
2,00001928-AE75-4AA9-8A47-4BDE96D08C2D,23040.0,0.0,0.0,11520.0,0.5,23040.0,1.0,23040.0,1.0,23040.0,1.0


In [62]:
sales_amount_ratio.to_csv('target.csv',index=False)

### 該品類(7,14,30,60,180天銷量,比例) 最大值 最小值 中位數 平均數

In [77]:
total_subtotal= orderdf.groupby('code')['subtotal'].sum().to_frame() #總銷售金額依檔次分組
total_subtotal= total_subtotal.rename(columns={'subtotal':'total_sales_amount'}).reset_index(drop=False)
total_subtotal.head(3)

,code,total_sales_amount
0,0.0,18506893.0
1,2.0,1515784.0
2,4.0,44808.0


In [80]:
##code
output=orderdf
list = [7,14,30,60,180]
code=pd.DataFrame()
for i in list:
    if(i==7):
        output_days=output[output.day_diff<i]  
        text=("sales_"+str(i)+"days")
        text2=("ratio_"+str(i)+"days")
        final_output = output_days.groupby('code')['subtotal'].sum().to_frame()  #i天數內之銷售金額
        final_output = final_output.rename(columns={'subtotal':text})  #更改欄位名稱
        total = pd.merge(total_subtotal,final_output ,how='left',on=['code'])
        total = total.fillna(0)   #補空值
        total[text2] = (total[text]/total['total_sales_amount']).round(3) #銷量占比
        code = total.reset_index(drop=True) 
    else:
        output_days=output[output.day_diff<i]  
        text=("sales_"+str(i)+"days")
        text2=("ratio_"+str(i)+"days")
        final_output = output_days.groupby('code')['subtotal'].sum().to_frame()  
        final_output = final_output.rename(columns={'subtotal':text})  
        final_output=final_output.reset_index() 
        total = pd.merge(total_subtotal,final_output ,how='left',on=['code'])
        total = total.fillna(0)   
        total[text2] = (total[text]/total['total_sales_amount']).round(3) 
        code=code.merge(total,how='left')

In [81]:
code.columns

Index(['code', 'total_sales_amount', 'sales_7days', 'ratio_7days',
       'sales_14days', 'ratio_14days', 'sales_30days', 'ratio_30days',
       'sales_60days', 'ratio_60days', 'sales_180days', 'ratio_180days'],
      dtype='object')

In [82]:
code.columns=['code', 'total_sales_amount', 'code_sales_7days', 'code_ratio_7days',
       'code_sales_14days', 'code_ratio_14days', 'code_sales_30days', 'code_ratio_30days',
       'code_sales_60days', 'code_ratio_60days', 'code_sales_180days', 'code_ratio_180days']
code.head(3)

,code,total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days
0,0.0,18506893.0,5865675.0,0.317,8728702.0,0.472,12569867.0,0.679,15237826.0,0.823,17362861.0,0.938
1,2.0,1515784.0,684725.0,0.452,970749.0,0.640,1187436.0,0.783,1247623.0,0.823,1382325.0,0.912
2,4.0,44808.0,1851.0,0.041,2631.0,0.059,4509.0,0.101,14144.0,0.316,32940.0,0.735


In [83]:
##code_max,min,median,mean
output=orderdf
list = [7,14,30,60,180]
for i in list:
    output_days=output[output.day_diff<i]  
    text=("code_max_"+str(i)+"days")
    text2=("code_min_"+str(i)+"days")
    text3=("code_median_"+str(i)+"days")
    text4=("code_mean_"+str(i)+"days")
    code=code.merge(output_days.groupby('code')['subtotal'].max().to_frame().reset_index().rename(columns={'subtotal':text}))
    code=code.merge(output_days.groupby('code')['subtotal'].min().to_frame().reset_index().rename(columns={'subtotal':text2}))
    code=code.merge(output_days.groupby('code')['subtotal'].mean().to_frame().reset_index().rename(columns={'subtotal':text3}))
    code=code.merge(output_days.groupby('code')['subtotal'].median().to_frame().reset_index().rename(columns={'subtotal':text4}))


In [84]:
code.columns=['code', 'code_total_sales_amount', 'code_sales_7days', 'code_ratio_7days',
       'code_sales_14days', 'code_ratio_14days', 'code_sales_30days',
       'code_ratio_30days', 'code_sales_60days', 'code_ratio_60days',
       'code_sales_180days', 'code_ratio_180days', 'code_max_7days',
       'code_min_7days', 'code_median_7days', 'code_mean_7days',
       'code_max_14days', 'code_min_14days', 'code_median_14days',
       'code_mean_14days', 'code_max_30days', 'code_min_30days',
       'code_median_30days', 'code_mean_30days', 'code_max_60days',
       'code_min_60days', 'code_median_60days', 'code_mean_60days',
       'code_max_180days', 'code_min_180days', 'code_median_180days',
       'code_mean_180days']
code.head(3)

,code,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days
0,0.0,18506893.0,5865675.0,0.317,8728702.0,0.472,12569867.0,0.679,15237826.0,0.823,17362861.0,0.938,9999.0,100.0,1028.524461,899.0,9999.0,100.0,1043.728566,899.0,16990.0,100.0,1103.200544,950.0,16990.0,100.0,1132.418698,989.0,16990.0,100.0,1140.118261,990.0
1,2.0,1515784.0,684725.0,0.452,970749.0,0.640,1187436.0,0.783,1247623.0,0.823,1382325.0,0.912,4038.0,149.0,541.712816,449.0,8076.0,149.0,555.030875,458.0,8076.0,99.0,560.375649,458.0,8076.0,99.0,561.739307,458.0,8076.0,99.0,569.326606,458.0
2,4.0,44808.0,1851.0,0.041,2631.0,0.059,4509.0,0.101,14144.0,0.316,32940.0,0.735,858.0,490.0,617.000000,503.0,858.0,390.0,526.200000,490.0,939.0,390.0,644.142857,503.0,1878.0,390.0,884.000000,898.5,1878.0,230.0,968.823529,939.0


In [85]:
code.to_csv('code_time.csv',index=False)

,parent_code,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days
0,2.0,1515784.0,684725.0,0.452,970749.0,0.640,1187436.0,0.783,1247623.0,0.823,1382325.0,0.912,4038.0,149.0,541.712816,449.0,8076.0,149.0,555.030875,458.0,8076.0,99.0,560.375649,458.0,8076.0,99.0,561.739307,458.0,8076.0,99.0,569.326606,458.0
1,4.0,44808.0,1851.0,0.041,2631.0,0.059,4509.0,0.101,14144.0,0.316,32940.0,0.735,858.0,490.0,617.000000,503.0,858.0,390.0,526.200000,490.0,939.0,390.0,644.142857,503.0,1878.0,390.0,884.000000,898.5,1878.0,230.0,968.823529,939.0
2,5.0,8517.0,844.0,0.099,1243.0,0.146,3724.0,0.437,4949.0,0.581,6921.0,0.813,445.0,399.0,422.000000,422.0,445.0,399.0,414.333333,399.0,729.0,363.0,465.500000,445.0,729.0,363.0,449.909091,445.0,729.0,363.0,461.400000,445.0
